## TfidfVectorizer를 이용해 벡터화 <br>
---------------------------------------------------
1) youtube data api로 검색 결과 가져오기 <br>
2) re 패키지를 이용해 가져온 검색 결과에서 특수문자 제거 <br>
3) 검색 결과 리스트를 tfidfvectorizer를 이용해 벡터화 <br>
4) 벡터화한 결과를 sentence bert(klue/roberta-base모델)의 입력에 맞게 변환한 뒤 넣어보기

In [2]:
!pwd

/content


In [1]:
# 드라이브에서 config 파일을 읽기 위해
from google.colab import drive   
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd drive/My Drive/Noggro/server

/content/drive/My Drive/Noggro/server


In [5]:
!pwd

/content/drive/My Drive/Noggro/server


In [6]:
# youtube data api를 위해 필요한 라이브러리 선언
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import config

In [7]:
API_KEY = config.youtube_api_key   # API Key
YOUTUBE_API_SERVICE_NAME="youtube"
YOUTUBE_API_VERSION="v3"
youtube = build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=API_KEY)

In [8]:
# 검색 결과 가져오기
search_response = youtube.search().list(
    q = "설현",   # 검색어 설현
    order = "relevance",    # 관련성 순으로 보여줌
    part = "snippet",
    maxResults = 30
).execute()

In [9]:
search_response

{'etag': '4BydRD9cpsaHUvgZhlfCAeufuT4',
 'items': [{'etag': 'Ihs6ZcG48ZkiqjYQi8MZoipj8c4',
   'id': {'kind': 'youtube#video', 'videoId': 'QyB7k6u61UE'},
   'kind': 'youtube#searchResult',
   'snippet': {'channelId': 'UC0mwtx068PEAw8-LvzRKiWQ',
    'channelTitle': '비하인드 ',
    'description': '연예인들의 알려지지 않은 비하인드 스토리를 소개합니다. #설현 #설현근황 Music: Escape - Atch ...',
    'liveBroadcastContent': 'none',
    'publishTime': '2022-07-28T11:10:05Z',
    'publishedAt': '2022-07-28T11:10:05Z',
    'thumbnails': {'default': {'height': 90,
      'url': 'https://i.ytimg.com/vi/QyB7k6u61UE/default.jpg',
      'width': 120},
     'high': {'height': 360,
      'url': 'https://i.ytimg.com/vi/QyB7k6u61UE/hqdefault.jpg',
      'width': 480},
     'medium': {'height': 180,
      'url': 'https://i.ytimg.com/vi/QyB7k6u61UE/mqdefault.jpg',
      'width': 320}},
    'title': '설현, 외모까지 바꿔놓은 희귀병 충격정체...설현의 현재상태가 심상치않은 이유ㄷㄷ (Feat.SNS)'}},
  {'etag': 'Kg5T-8d8fQ-IsbisI570KTksLSk',
   'id': {'channelId': 'UC0wIGRaeKQa45

In [10]:
# 영상 title, description, thumbnail등의 정보는 items항목에 있으므로 items가져옴
video_json = {}
json_index = 0


for item in search_response['items']:
    if item['id']['kind'] == 'youtube#video':  # video 항목에 대해서만 가져 옴
        video_json[json_index] = {"videoId": item['id']['videoId'], 
                                  "title": item['snippet']['title'], 
                                  "desc" : item['snippet']['description'], 
                                  "thumbnail": item['snippet']['thumbnails']['medium']['url']}
    
        json_index += 1 
    
# json을 데이터프레임으로 변환
videoId = []
title = []
desc = []
thumbnail = []

for i in range(len(video_json)):
    videoId.append(video_json[i]['videoId'])
    title.append(video_json[i]['title'])
    desc.append(video_json[i]['desc'])
    thumbnail.append(video_json[i]['thumbnail'])
    
df = pd.DataFrame({"videoId": videoId,
                   "title": title,
                   "desc": desc,
                   "thumbnail": thumbnail})

In [11]:
df

,videoId,title,desc,thumbnail
0,QyB7k6u61UE,"설현, 외모까지 바꿔놓은 희귀병 충격정체...설현의 현재상태가 심상치않은 이유ㄷㄷ ...",연예인들의 알려지지 않은 비하인드 스토리를 소개합니다. #설현 #설현근황 Music...,https://i.ytimg.com/vi/QyB7k6u61UE/mqdefault.jpg
1,ZVDXaxlKwIk,HOT한 알바생☆ 손님들의 사랑을 한 몸에 받는 설현♡ #highlight #어쩌다...,티빙에서 스트리밍 : https://tving.onelink.me/xHqC/30a7...,https://i.ytimg.com/vi/ZVDXaxlKwIk/mqdefault.jpg
2,RSYxuL2X8ow,설현 또 부작용 호소,모든 영상은 저의 개인적인 의견이 포함되어 있습니다. 시청에 주의해주세요. [후원하...,https://i.ytimg.com/vi/RSYxuL2X8ow/mqdefault.jpg
3,64hE5RL_VHM,"5년 만에 불편한 만남, 지코가 설현 만날때 했던 짓",,https://i.ytimg.com/vi/64hE5RL_VHM/mqdefault.jpg
4,d9FgG-6ODcI,박명수한테 꿀밤맞고 우는 설현,박명수한테 꿀밤맞고 우는 설현 출처 : 굿스토리.,https://i.ytimg.com/vi/d9FgG-6ODcI/mqdefault.jpg
5,o49aI2RUXdU,[#어쩌다사장2] 마스크에 가려져 아무도 못 알아본 설현ㅋㅋㅋ 오자마자 카운터 분위...,어쩌다사장2 #Diggle #디글 취향저격! 당신이 원하던 그 플레이리스트 [#디제...,https://i.ytimg.com/vi/o49aI2RUXdU/mqdefault.jpg
6,DQ75NPJsqNo,지코가 박신혜 결혼식장에서 설현을 마주치고 보인 반응,"드라마의 모든것, 드라마갤러리입니다. 드라마, 영화, 배우들의 비하인드 스토리를 다...",https://i.ytimg.com/vi/DQ75NPJsqNo/mqdefault.jpg
7,ASAKtRUOfV0,【분량체크】 아이돌 설현에서 배우 김설현✨ 이상형 송중기와 꼭 작품을 함께하고 싶다...,이 영상은 편집본입니다* 라디오스타 635회 - [넌 is 뭔들] 특집 (남궁민.이...,https://i.ytimg.com/vi/ASAKtRUOfV0/mqdefault.jpg
8,kZe3Lk_ZmnY,백신 2차 접종 24시간의 기록💉 | 백신 브이로그 | 설현 Vlog | 눈이 부시...,"얼마 전 2차 백신을 접종하고 왔는데요~ 백신 맞으러 가기 전 준비하는 모습부터, ...",https://i.ytimg.com/vi/kZe3Lk_ZmnY/mqdefault.jpg
9,-UTUTtxc-C8,"&#39;살인자의 쇼핑목록&#39; 설현, 당신이 몰랐던 11가지 사실",설현 #살인자의쇼핑목록 #어쩌다사장 '살인자의 쇼핑목록'과 '어쩌다사장2' 출연중인...,https://i.ytimg.com/vi/-UTUTtxc-C8/mqdefault.jpg


In [30]:
# 2) 검색 결과 리스트에서 특수문자 제거
import re

title_token = title
for i in range(len(title_token)):
    title_token[i] = re.sub(r'[^\w]', ' ', title_token[i])

desc_token = desc
for i in range(len(desc_token)):
    desc_token[i] = re.sub(r'[^\w]', ' ', desc_token[i])

In [31]:
title_token

['설현  외모까지 바꿔놓은 희귀병 충격정체   설현의 현재상태가 심상치않은 이유ㄷㄷ  Feat SNS ',
 'HOT한 알바생  손님들의 사랑을 한 몸에 받는 설현   highlight  어쩌다사장2 EP 9',
 '설현 또 부작용 호소',
 '5년 만에 불편한 만남  지코가 설현 만날때 했던 짓',
 '박명수한테 꿀밤맞고 우는 설현',
 '  어쩌다사장2  마스크에 가려져 아무도 못 알아본 설현ㅋㅋㅋ 오자마자 카운터 분위기 확 살려버린 만능 설현 효과     디제이픽',
 '지코가 박신혜 결혼식장에서 설현을 마주치고 보인 반응',
 ' 분량체크  아이돌 설현에서 배우 김설현  이상형 송중기와 꼭 작품을 함께하고 싶다는 바램  그런데 박명수랑은 무슨 사이     라디오스타   TVPP   MBC 160330 방송',
 '백신 2차 접종 24시간의 기록    백신 브이로그   설현 Vlog   눈이 부시게 by 설현',
 '  39 살인자의 쇼핑목록  39  설현  당신이 몰랐던 11가지 사실',
 ' Korean Music Wave  Hani  Seolhyun  Tzuyu   Tell Me   Nobody  하니  설현  쯔위    텔미   노바디 20161009',
 '  인기급상승  투닥투닥 거리는 모습도 예쁜 예비 부부  설현 보고 사레 들린 남편을 향한 예비 아내의 귀여운 질투    어쩌다사장2',
 '161231 AOA 설현 Sulhyun   39 내 남자친구에게 To my boyfriend  39  4K 직캠  MBC가요대제전 by DaftTaengk',
 '광고계 블루칩 AOA설현 레전드 CF 10선',
 '내가 요즘 먹는 것  feat ASMR    설현 Vlog   눈이 부시게 by 설현',
 'AOA 설현  167cm 48kg 체중관리 걸그룹 그만둘까 고민  HIT  해피투게더3  20150212',
 '연쇄살인마 잡아서 결혼자금 마련하려는 광수와 설현의  살인자의 쇼핑목록 ',
 ' SUB  설현  서강준 훔쳐보며 두근두근  완전 팬이에요

In [32]:
desc_token

['연예인들의 알려지지 않은 비하인드 스토리를 소개합니다   설현  설현근황 Music  Escape   Atch    ',
 '티빙에서 스트리밍   https   tving onelink me xHqC 30a78d6f  차태현  조인성 어쩌다 사장2 Unexpected Business2 EP 9  어쩌다     ',
 '모든 영상은 저의 개인적인 의견이 포함되어 있습니다  시청에 주의해주세요   후원하기  후원계좌  국민은행 619202 04 173003    ',
 '',
 '박명수한테 꿀밤맞고 우는 설현 출처   굿스토리 ',
 '어쩌다사장2  Diggle  디글 취향저격  당신이 원하던 그 플레이리스트   디제이픽    Let s giggle   Diggle  방송국놈들이 덕질하는    ',
 '드라마의 모든것  드라마갤러리입니다  드라마  영화  배우들의 비하인드 스토리를 다룹니다   설현  지코  AOA ',
 '이 영상은 편집본입니다  라디오스타 635회    넌 is 뭔들  특집  남궁민 이동휘 설현 윤정수   설현  SEOLHYUN  라디오스타  TVPP ',
 '얼마 전 2차 백신을 접종하고 왔는데요  백신 맞으러 가기 전 준비하는 모습부터  백신 접종 후 24시간까지의 생생한 후기를 기록해    ',
 '설현  살인자의쇼핑목록  어쩌다사장  살인자의 쇼핑목록 과  어쩌다사장2  출연중인 설현에 대해서 알아보겠습니다 이상이다 ',
 'Hani  Seolhyun  Tzuyu   Tell Me   Nobody  하니  설현  쯔위    텔미   노바디 2016 DMC Festival Korean Music Wave 20161009    ',
 '어쩌다사장2  Diggle  디글 유튜브에서 지금 핫한 사람 누구  人기가 급상승한 人물 모음집 Let s giggle   Diggle  방송국놈들이 덕질    ',
 '161231 AOA 설현 Sulhyun  내 남자친구에게 To my boyfriend  4K 직캠  MBC가요대제전 by DaftTaen

In [15]:
# 3) tfidfvectorizer 이용해서 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_title = TfidfVectorizer()  
tfidf_title_fit_trans = tfidf_title.fit_transform(title_token)

tfidf_desc = TfidfVectorizer()
tfidf_desc_fit_trans = tfidf_desc.fit_transform(desc_token)

print(tfidf_title_fit_trans.shape)  # (25, 225), 225은 25개의 제목에 있는 단어들을 분리한 개수
print(tfidf_desc_fit_trans.shape)

(26, 236)
(26, 268)


In [16]:
tfidf_title.vocabulary_

{'10선': 0,
 '11가지': 1,
 '160330': 2,
 '161231': 3,
 '167cm': 4,
 '1도': 5,
 '20150212': 6,
 '20160415': 7,
 '20161009': 8,
 '209회': 9,
 '211113': 10,
 '24시간의': 11,
 '2차': 12,
 '306회': 13,
 '39': 14,
 '48kg': 15,
 '4k': 16,
 '5년': 17,
 'amp': 18,
 'aoa': 19,
 'aoa설현': 20,
 'asmr': 21,
 'boyfriend': 22,
 'bros': 23,
 'by': 24,
 'cf': 25,
 'cover': 26,
 'coverㅣ눈이': 27,
 'dafttaengk': 28,
 'diggle': 29,
 'ep': 30,
 'feat': 31,
 'hani': 32,
 'highlight': 33,
 'hit': 34,
 'hot한': 35,
 'hyun': 36,
 'iu': 37,
 'jtbc': 38,
 'knowing': 39,
 'korean': 40,
 'mbc': 41,
 'mbc가요대제전': 42,
 'me': 43,
 'moment': 44,
 'music': 45,
 'my': 46,
 'nobody': 47,
 'seol': 48,
 'seolhyun': 49,
 'sexy': 50,
 'sns': 51,
 'sub': 52,
 'sulhyun': 53,
 'tell': 54,
 'to': 55,
 'tvpp': 56,
 'tzuyu': 57,
 'vlog': 58,
 'wave': 59,
 'welcome': 60,
 'xiumin': 61,
 'ㅣ설현': 62,
 '가려져': 63,
 '가수': 64,
 '갓구운클립': 65,
 '거리는': 66,
 '걸그룹': 67,
 '결혼식장에서': 68,
 '결혼자금': 69,
 '경력직들': 70,
 '고민': 71,
 '고충': 72,
 '관계의': 73,
 '광고계': 74,
 '광수

In [17]:
title_sim = tfidf_title_fit_trans.toarray() 

In [18]:
tfidf_matrix = pd.DataFrame(
    title_sim, 
    columns=tfidf_title.get_feature_names_out()
)

desc_sim = tfidf_desc_fit_trans.toarray()
desc_matrix = pd.DataFrame(
    desc_sim,
    columns=tfidf_desc.get_feature_names_out()
)

In [19]:
tfidf_matrix

,10선,11가지,160330,161231,167cm,1도,20150212,20160415,20161009,209회,...,해피투게더3,했던,향한,현재상태가,형님,호소,홍일점,효과,훔쳐보며,희귀병
0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.325055,0.000000,0.0000,0.000000,0.000000,0.000000,0.325055
1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.6867,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.342962,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.284744,0.000000,0.000000
6,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.22602,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.347007,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [20]:
maxidx = tfidf_matrix.idxmax()
maxidx

10선       13
11가지       9
160330     7
161231    12
167cm     15
          ..
호소         2
홍일점       21
효과         5
훔쳐보며      17
희귀병        0
Length: 236, dtype: int64

In [21]:
%cd ../

/content/drive/MyDrive/Noggro


In [22]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.4 MB/s 
     |████████████████████████████████| 4.7 MB 26.4 MB/s 
     |████████████████████████████████| 1.2 MB 55.5 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=053865852262858f4c1c62e2f4b7e6904c2d89dabf1e17408929781a3a3f312e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [25]:
# 훈련한 모델 import
import joblib
import torch
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util

In [24]:
my_model = joblib.load('./myklue-roberta-base.pkl')

In [33]:
embedding = my_model.encode(title_token)

embedding2 = my_model.encode(desc_token)
    
top_k = len(title_token)

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(embedding2, embedding)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

title_list = []
desc_list = []
list_score = []

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    title_list.append(title_token[idx])
    desc_list.append(desc_token[idx])
    list_score.append(float(score))
    
title_desc_df = pd.DataFrame(list(zip(title_list, desc_list, list_score)), columns=['title', 'desc', 'score'])

In [34]:
title_desc_df

,title,desc,score
0,Korean Music Wave Hani Seolhyun Tzuyu Te...,Hani Seolhyun Tzuyu Tell Me Nobody 하니 ...,0.470970
1,AOA Seolhyun Sexy Moment 설현 에이오에이 섹시한 순간,Check Out My Website http kpopict site htt...,0.457040
2,내가 요즘 먹는 것 feat ASMR 설현 Vlog 눈이 부시게 by 설현,오늘은 제가 요즘 먹는 것들에 대해서 소개하려고 합니다 특별한 건 없지만 아주 간...,0.398571
3,161231 AOA 설현 Sulhyun 39 내 남자친구에게 To my boyf...,161231 AOA 설현 Sulhyun 내 남자친구에게 To my boyfrien...,0.382188
4,어쩌다사장2 마스크에 가려져 아무도 못 알아본 설현ㅋㅋㅋ 오자마자 카운터 분위...,어쩌다사장2 Diggle 디글 취향저격 당신이 원하던 그 플레이리스트 디제...,0.374061
5,설현 지민 나락의 순간,AOA 설현 지민 긴또깡 논란,0.367284
6,39 그 사람 IU 39 Cover by SEOLHYUN ㅣ설현 Cov...,정말 오랜만에 세 번째 커버를 들고 왔어요 풍경과 함께 힐링의 시간이 되길 바라요...,0.365340
7,설현 외모까지 바꿔놓은 희귀병 충격정체 설현의 현재상태가 심상치않은 이유ㄷㄷ ...,연예인들의 알려지지 않은 비하인드 스토리를 소개합니다 설현 설현근황 Music...,0.363537
8,어쩌다사장2 배우와 가수 모두 해내야 했던 설현의 고충 편안한 술자리 분위...,유퀴즈온더블럭 요즘 뜨는 핫클립 지금 생성 중 샾잉 샾잉 최신 콘텐츠 보러...,0.342810
9,AOA 설현 167cm 48kg 체중관리 걸그룹 그만둘까 고민 HIT 해피투게...,이날 설현은 자신을 먹방중독 이라고 소개하며 먹는 걸 너무 좋아해 다이어트가 힘...,0.337416
